In [1]:
import numpy as np
import cv2

from scipy import ndimage
import glob

import SimpleITK as sitk

In [ ]:
def resize_images_3d(img,desired_depth,desired_width,desired_height,current_depth,current_width,current_height,order): # img is 1024*1024
    
    width_index=current_width/desired_width
    height_index=current_height/desired_height
    depth_index=current_depth/desired_depth

    depth_factor=1/depth_index
    width_factor=1/width_index
    height_factor=1/height_index
    
    img=ndimage.zoom(img,(depth_factor,width_factor,height_factor),order=order)
    
    return img

def imcrop(img):
    
    if len(img.shape) > 2:
        z, y, x = img.shape
        v = min(x,y,z)
    else:
        y, x = img.shape
        v = min(x,y)
            
    
    if len(img.shape) > 2:
        new_x=v
        new_y=v
        new_z=v

        left = int((y - new_y)/2)
        top = int((x - new_x)/2)
        right = int((y + new_y)/2)
        bottom = int((x + new_x)/2)
        z_top = int((z - new_z)/2)
        z_bottom = int((z + new_z)/2)
        
        k = img[z_top:z_bottom,left:right,top:bottom]
    else:
        new_x=v
        new_y=v

        left = int((y - new_y)/2)
        top = int((x - new_x)/2)
        right = int((y + new_y)/2)
        bottom = int((x + new_x)/2)
        
        k = img[left:right,top:bottom]
    
    return k

def process3d(img3d,ax_aspect,sag_aspect,cor_aspect,order):
    
    img3d = resize_images_3d(img3d,int(img3d.shape[0]*cor_aspect),int(img3d.shape[1]*ax_aspect),int(img3d.shape[2]),img3d.shape[0],img3d.shape[1],img3d.shape[2],order)
       
    if order > 0:
        img3d = (img3d - img3d.min()).astype('uint16')
    
    img3d = imcrop(img3d)
    
    img3d = resize_images_3d(img3d,1024,1024,1024,img3d.shape[0],img3d.shape[1],img3d.shape[2],order)
    
    return img3d

def process2d(img3d):
    
    N = img3d.shape[1]
    
    img2d = np.sum(img3d,axis=1)

    img2d=ndimage.rotate(img2d,180,reshape=False) # rotate the image to 90
    img2d = np.fliplr(img2d)
    img2d = (img2d/N).astype('float32')
    
    return img2d

def transform(I, Ic, Isd):
    
    a = 1.0
    b = 0.7
    
    if Ic==0 and Isd==0:
        Ic = I.mean()
        Isd = I.std()

    h = 1/(1+np.exp(-a*(I-b*Ic)/Isd))
    
    return h, Ic, Isd

def load_itk(filename):
    # Reads the image using SimpleITK
    itkimage = sitk.ReadImage(filename)

    # Convert the image to a  numpy array first and then shuffle the dimensions to get axis in the order z,y,x
    ct_scan = sitk.GetArrayFromImage(itkimage)

    # Read the origin of the ct_scan, will be used to convert the coordinates from world to voxel and vice versa.
    origin = np.array(list(reversed(itkimage.GetOrigin())))

    # Read the spacing along each dimension
    spacing = np.array(list(reversed(itkimage.GetSpacing())))

    return ct_scan, origin, spacing

def sharp(img,L=1):
    
    if L==1:
        Wu = 3
    else:
        Wu = 1.5
    
    kernel1 = np.ones((25,25), np.float32)/625
    
    s_img = cv2.filter2D(img,-1,kernel1)
    
    
    img = img + Wu*(img-s_img)

    img[img<=0]=0
    return img


# LUNA16

In [3]:
LUNA16_path = "./LUNA16/CT/CT_raw/*.mhd"

LUNA16_mask_path = "./LUNA16/CT/seg-lungs-LUNA16/"

input_path = "./LUNA16/CT/Xray/"
lung_path = "./LUNA16/CT/Lungs/"


CT_files = []
for name in glob.glob(LUNA16_path):
    #print("loading: {}".format(name))
    CT_files.append(name)
print(len(CT_files))

1


In [4]:
CT_threshold = 128

i=0

for x in range(0,len(CT_files)):

    ct_scan, origin, spacing = load_itk(CT_files[x])

    if ct_scan.shape[0]>CT_threshold:

        CT_mask_files = LUNA16_mask_path + CT_files[x][19:] 

        mask_ct_scan, _, _ = load_itk(CT_mask_files)

        CT_img3d = ct_scan
        mask_ct_scan = mask_ct_scan

        sz, sy, sx = spacing

        # pixel aspects, assuming all slices are the same
        ps = [sx , sy]
        ss = sz
        ax_aspect = ps[1]/ps[0]
        sag_aspect = ps[1]/ss
        cor_aspect = ss/ps[0]


        #mask

        Mask_img3d = mask_ct_scan

        Mask_img3d = Mask_img3d > 0

        CT_img3d[CT_img3d<-1024]=-1024

        Bone_img3d = (CT_img3d>205*1).astype(np.uint8)

        kernel = np.ones((5, 5), np.uint8)

        Bone_img3d = cv2.morphologyEx(Bone_img3d, cv2.MORPH_CLOSE, kernel)

        Bone_img3d = process3d(Bone_img3d,ax_aspect,sag_aspect,cor_aspect, order=0)

        CT_img3d = process3d(CT_img3d,ax_aspect,sag_aspect,cor_aspect, order=2)

        Mask_img3d = process3d(Mask_img3d,ax_aspect,sag_aspect,cor_aspect, order=0)
        Mask_img2d = process2d(Mask_img3d)
        Mask_img2d[Mask_img2d>0] = 1

        Lungs_img3d = np.multiply(CT_img3d, Mask_img3d)

        Bone_img3d = np.multiply(CT_img3d, Bone_img3d)

        Soft_img3d = CT_img3d - Lungs_img3d

        CT_img2d = process2d(CT_img3d)
        CT_img2d, Ic, Isd = transform(CT_img2d, 0, 0)

        Soft_img2d = process2d(Soft_img3d)
        Soft_img2d, Ic, Isd = transform(Soft_img2d, Ic, Isd)

        Bone_img2d = process2d(Bone_img3d)
        Bone_img2d, Ic, Isd = transform(Bone_img2d, 0, 0)

        Lungs_img2d = CT_img2d - Soft_img2d

        Soft_img2d = Soft_img2d + 0.5*Bone_img2d
        
        Soft_img2d = sharp(Soft_img2d,1)
        Lungs_img2d = sharp(Lungs_img2d,0)
        
        Lungs_img2d = Lungs_img2d*Mask_img2d
        
        CT_img2d = Soft_img2d + Lungs_img2d
        
        i = i + 1
        
        np.save(input_path+'/Luna16_Xray_5b_'+str(i),CT_img2d)
        np.save(lung_path+'/Luna16_Lungs_5b_'+str(i),Lungs_img2d)

        print("Saved data_" + str(x) + " count: " + str(i))
        
    else:
        print("Unsaved data index: " + str(x))

Saved data_0 count: 1
